<a href="https://colab.research.google.com/github/Madhuri1506/CXR-FL1/blob/main/test3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import torch
from torch.utils.tensorboard import SummaryWriter

In [2]:
#do once as files were  uploaded
from google.colab import files

# Upload the zip file
uploaded = files.upload()


Saving src.zip to src (1).zip


In [ ]:
import zipfile
import io

# Extract the contents of the zip file
with zipfile.ZipFile(io.BytesIO(uploaded['src.zip']), 'r') as zip_ref:
    zip_ref.extractall()


In [5]:
from src.update_numeric import NumericModelUpdate
from src.strategy import FedAvg
from src.test import test_numeric


In [6]:
# Load your numeric datasets (replace placeholders with actual paths or URLs)
ieee_data = pd.read_csv('/content/drive/MyDrive/UCI_5DB_918_12/heart_statlog_cleveland_hungary_final.csv')
fhs_data = pd.read_csv('/content/drive/MyDrive/UCI_5DB_918_12/CHD_preprocessed.csv')

# Preprocess the datasets (replace with your preprocessing steps)
# For simplicity, we assume there are no missing values in this example
# Also, you may need more sophisticated preprocessing depending on your datasets


In [8]:
ieee_data.head()

,age,sex,chest pain type,resting bp s,cholesterol,fasting blood sugar,resting ecg,max heart rate,exercise angina,oldpeak,ST slope,target
0,40,1,2,140,289,0,0,172,0,0.0,1,0
1,49,0,3,160,180,0,0,156,0,1.0,2,1
2,37,1,2,130,283,0,1,98,0,0.0,1,0
3,48,0,4,138,214,0,0,108,1,1.5,2,1
4,54,1,3,150,195,0,0,122,0,0.0,1,0


In [9]:
fhs_data.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,1,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,1,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,1,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [10]:
fhs_data=fhs_data.rename(columns={'TenYearCHD':'target'})

In [11]:
fhs_data.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,target
0,1,39,1,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,1,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,1,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [12]:
def preprocess_numeric_data(data):
  X = data.drop('target', axis=1) # Replace 'target_column' with the actual target column name
  y = data['target']
 # Split the data into train and test sets
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Standardize the features
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train)
  X_test_scaled = scaler.transform(X_test)

  return X_train_scaled, y_train, X_test_scaled, y_test

In [13]:
# Preprocess IEEE dataset
X_train_ieee, y_train_ieee, X_test_ieee, y_test_ieee = preprocess_numeric_data(ieee_data)

# Preprocess FHS dataset
X_train_fhs, y_train_fhs, X_test_fhs, y_test_fhs = preprocess_numeric_data(fhs_data)


In [14]:
# Define your numeric model architecture (replace with your model definition)
class NumericModel(torch.nn.Module):
   def __init__(self, input_size, hidden_size, output_size):
      super(NumericModel, self).__init__()
      self.fc1 = torch.nn.Linear(input_size, hidden_size)
      self.relu = torch.nn.ReLU()
      self.fc2 = torch.nn.Linear(hidden_size, output_size)

      def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Set up your Federated Learning parameters
# (replace with your specific configuration)
class FedArgs:
    def __init__(self):
        self.num_users = 10
        self.rounds = 10
        self.frac = 0.1
# Add other relevant parameters for your setup

args_ieee = FedArgs()# Set your specific parameters for the IEEE dataset
args_fhs = FedArgs()  # Set your specific parameters for the FHS dataset

# Set up your model instances
input_size = X_train_ieee.shape[1] # Replace with your actual feature dimension
hidden_size = 64 # Adjust as needed
output_size = 2  # Replace with the number of classes in your target variable

model_ieee = NumericModel(input_size, hidden_size, output_size)
model_fhs = NumericModel(input_size, hidden_size, output_size)

# Set up your TensorBoard writers
writer_ieee = SummaryWriter(log_dir='logs/ieee_experiment')
writer_fhs = SummaryWriter(log_dir='logs/fhs_experiment')

# Perform Federated Learning for IEEE dataset
# (replace with your specific Federated Learning loop)
for iter in range(args_ieee.rounds):
 # Implement your Federated Learning logic using model_ieee and data from X_train_ieee, y_train_ieee

# Log metrics to TensorBoard (replace with your actual metrics)
    accuracy_ieee = np.random.uniform(0.7, 0.9)
    loss_ieee = np.random.uniform(0.1, 0.3)
    writer_ieee.add_scalar("Test accuracy", accuracy_ieee, iter)
    writer_ieee.add_scalar("Test loss", loss_ieee, iter)
# Perform Federated Learning for FHS dataset
# (replace with your specific Federated Learning loop)
for iter in range(args_fhs.rounds):
 # Implement your Federated Learning logic using model_fhs and data from X_train_fhs, y_train_fhs

 # Log metrics to TensorBoard (replace with your actual metrics)
    accuracy_fhs = np.random.uniform(0.6, 0.8)
    loss_fhs = np.random.uniform(0.2, 0.4)
    writer_fhs.add_scalar("Test accuracy", accuracy_fhs, iter)
    writer_fhs.add_scalar("Test loss", loss_fhs, iter)

# Close TensorBoard writers


In [15]:
# pip install -U syft

In [16]:
import syft as sy
sy.requires(">=0.8.2,<0.8.3")
# domain_client = sy.login(port=8080, email="info@openmined.org", password="changethis")

✅ The installed version of syft==0.8.2 matches the requirement >=0.8.2 and the requirement <0.8.3


In [17]:
# Create a TorchHook
hook = sy.TorchHook(torch)


AttributeError: ignored